In [1]:
import numpy as np
import pandas as pd
# import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from scipy.stats import norm
# Load the pre-trained model
model = load_model('drought_prediction_model.keras')

# Read the dataset
# df = pd.read_csv("Data_set.csv")
# df['Date'] = pd.to_datetime(df['Date'])

# Read daily statistics from CSV file
daily_stats = pd.read_csv('daily_statistics.csv')

# Function to calculate SPEI for one day
def calculate_spei_one_day(day_of_year, precipitation, evaporation, stats):
    ref_precip_mean = stats.loc[stats['DayOfYear'] == day_of_year, 'Precip_Mean'].values[0]
    ref_precip_std = stats.loc[stats['DayOfYear'] == day_of_year, 'Precip_Std'].values[0]
    ref_evap_mean = stats.loc[stats['DayOfYear'] == day_of_year, 'Evap_Mean'].values[0]
    ref_evap_std = stats.loc[stats['DayOfYear'] == day_of_year, 'Evap_Std'].values[0]

    # Compute standardized anomalies
    precip_anomaly = (precipitation - ref_precip_mean) / ref_precip_std
    evap_anomaly = (evaporation - ref_evap_mean) / ref_evap_std

    # Combine the anomalies
    D_anomaly = precip_anomaly - evap_anomaly

    # Convert the anomaly to a standardized SPEI value using the inverse normal CDF (probit function)
    spei_value = norm.ppf(norm.cdf(D_anomaly))

    return spei_value

# Load the pre-trained model
model = load_model('drought_prediction_model.keras')

# Read the dataset
df = pd.read_csv("Data_set.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Select relevant columns
dff = df[['Date', 'MaxT', 'MinT', 'RH1', 'RH2', 'Wind', 'Rain', 'Evap', 'Radiation', 'Lat', 'Lon', 'Cum_Rain']]

# Fix specific data point
X = dff.drop('Date', axis=1)

# Scale the features
scaler = MinMaxScaler()
scaler.fit(X)

inp = X.tail(7).to_numpy()
inp_norm = scaler.transform(inp)
serri = inp_norm.reshape(1, 7, 11)

# Predict drought using the model
predictions = []
spei_one_day = []
# Output predictions and SPEI for each date
start_date = pd.to_datetime('2018-09-01')
end_date = pd.to_datetime('2018-09-30')
date_range = pd.date_range(start_date, end_date, freq='D')


current_date = date_range[0]
day_of_year = current_date.dayofyear
predict = model.predict(serri)
pre_inv = scaler.inverse_transform(predict)
predictions.append(pre_inv)
np.concatenate((inp[1:], pre_inv), axis=0)
precipitation_day = predictions[0][0][5]  # Assuming 'Rain' is the 6th column
evaporation_day = predictions[0][0][6]    # Assuming 'Evap' is the 7th column
spei_value = calculate_spei_one_day(day_of_year, precipitation_day, evaporation_day, daily_stats)
spei_one_day.append(spei_value)
print(len(spei_one_day))

# Calculate SPEI for one day


for i in range(29):
    

    # Get day of year for the current prediction dateas
    current_date = date_range[i+1]
    day_of_year = current_date.dayofyear
    
    #predict
    predict = model.predict(serri)
    pre_inv = scaler.inverse_transform(predict)
    
    # Calculate SPEI for the new data point
    predictions.append(pre_inv)
    np.concatenate((inp[1:], pre_inv), axis=0)
    precipitation_day = predictions[0][0][5]  # Assuming 'Rain' is the 6th column
    evaporation_day = predictions[0][0][6]    # Assuming 'Evap' is the 7th column
    spei_value = calculate_spei_one_day(day_of_year, precipitation_day, evaporation_day, daily_stats)
    spei_one_day.append(spei_value)
    print(len(spei_one_day))


p_date = {}
print(predictions[0])
for i in range(len(predictions)):
    p_date[date_range[i]] = {"prediction": predictions[i], "spei": spei_one_day[i]}

for date, values in p_date.items():
    print(date, values)

C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
print(spei_one_day[0])

In [43]:
print(spei_one_day)

[-0.5400480480586962, -0.9004765665765433, -0.7911125938167203, -0.6671905873434147, -0.6281535303644749, -0.927883713858435, -0.7144256517610627]


In [61]:
print(len(spei_one_day))

8


In [60]:
print(len(date_range))

8


In [59]:
print(len(predictions))

8


In [47]:
print(type(date_range))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [48]:
print(type(df['Date']))

<class 'pandas.core.series.Series'>


In [49]:
day_of_year

251

In [50]:
print(type(day_of_year))

<class 'int'>


In [1]:
pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show keras

Name: keras
Version: 2.15.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: 
Required-by: tensorflow-intel
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade tensorflow keras

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/8d/44/c604ecc5c9993b6574a681f2f505e980725871a89cfd9e48597b12ccb506/keras-3.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for h5py>=3.10.0 from https://files.pythonhosted.org/packages/d8/5e/b7b83cfe60504cc4d24746aed04353af7ea8ec104e597e5ae71b8d0390cb/h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for ml-dtypes~=0.3.1 from https://files.pythonhosted.org/packages/a4/db/1784b87285588788170f87e987bfb4bda218d

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Hp\\anaconda3\\Lib\\site-packages\\~l_dtypes\\_custom_floats.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



   -------------------------- ----------- 261.4/377.0 MB 325.2 kB/s eta 0:05:56
   -------------------------- ----------- 261.5/377.0 MB 324.8 kB/s eta 0:05:56
   -------------------------- ----------- 261.5/377.0 MB 324.6 kB/s eta 0:05:56
   -------------------------- ----------- 261.6/377.0 MB 325.3 kB/s eta 0:05:55
   -------------------------- ----------- 261.6/377.0 MB 324.8 kB/s eta 0:05:56
   -------------------------- ----------- 261.6/377.0 MB 325.5 kB/s eta 0:05:55
   -------------------------- ----------- 261.7/377.0 MB 325.8 kB/s eta 0:05:54
   -------------------------- ----------- 261.7/377.0 MB 325.8 kB/s eta 0:05:54
   -------------------------- ----------- 261.9/377.0 MB 336.3 kB/s eta 0:05:43
   -------------------------- ----------- 262.3/377.0 MB 356.6 kB/s eta 0:05:22
   -------------------------- ----------- 262.6/377.0 MB 361.5 kB/s eta 0:05:17
   -------------------------- ----------- 262.7/377.0 MB 361.5 kB/s eta 0:05:17
   -------------------------- ----------

In [4]:
pip show keras


Name: keras
Version: 2.15.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: 
Required-by: tensorflow-intel
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip show tensorflow



SyntaxError: invalid syntax (1727983140.py, line 1)

In [1]:
pip install --upgrade tensorflow keras

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/8d/44/c604ecc5c9993b6574a681f2f505e980725871a89cfd9e48597b12ccb506/keras-3.3.3-py3-none-any.whl.metadata
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for h5py>=3.10.0 from https://files.pythonhosted.org/packages/d8/5e/b7b83cfe60504cc4d24746aed04353

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Hp\\anaconda3\\Lib\\site-packages\\~5py\\defs.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
pip install tensorflow==2.16.1 keras==3.3.3


  Obtaining dependency information for tensorflow==2.16.1 from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Obtaining dependency information for keras==3.3.3 from https://files.pythonhosted.org/packages/8d/44/c604ecc5c9993b6574a681f2f505e980725871a89cfd9e48597b12ccb506/keras-3.3.3-py3-none-any.whl.metadata
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/87/67/a37f6214d0e9fe57f6ae54b

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Hp\\anaconda3\\Lib\\site-packages\\~ensorflow\\compiler\\mlir\\quantization\\tensorflow\\python\\pywrap_quantize_model.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
pip show keras


Name: kerasNote: you may need to restart the kernel to use updated packages.

Version: 3.3.3
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow-intel


In [4]:
pip show tensorflow


Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow==2.16.1


  Obtaining dependency information for tensorflow==2.16.1 from https://files.pythonhosted.org/packages/e4/14/d795bb156f8cc10eb1dcfe1332b7dbb8405b634688980aa9be8f885cc888/tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [6]:
pip show tensorflow

Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Hp\anaconda3\Lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.
